In [2]:
import urllib.request, json, datetime, time,pytz
import requests
from requests.structures import CaseInsensitiveDict
with open('keys.json') as json_file:
    keys = json.load(json_file)
trainkey = keys["trainkey"]
buskey = keys["buskey"]

#CTA bus and train trackers require an API key from the CTA.
#the following links have how to get an API key for bus and train trackers:
#https://www.transitchicago.com/developers/bustracker/
#https://www.transitchicago.com/developers/ttdocs/

#see the keys_sample.json file for proper formatting

In [17]:
def fetchCTAtrains(trainstop):
    if trainstop == "":
        return
    train_url = "http://lapi.transitchicago.com/api/1.0/ttarrivals.aspx?key="+trainkey+"&mapid="+trainstop+"&outputType=JSON"
    print(train_url)
    with urllib.request.urlopen(train_url) as url:
        train_info = json.loads(url.read().decode())
    return train_info
    #this pulls data from the CTA API.  The input is the ID for the stop, which must be a string.
    #this needs to be the "parent" stop ID, not the stop ID for the specific service.

def parseCTAtime(timestamp):
    return datetime.datetime.strptime(timestamp,"%Y-%m-%dT%H:%M:%S")
    #turns the CTA's timestamp into timezone naive datetime objects

In [26]:
def CTAtraininfo(traindata):

    if traindata == None:
        return [],[]
    output = ""
    #process all trains at given stop
    trainsatstop = []
    for train in traindata["ctatt"]["eta"]:
        trainsatstop.append(train)
        #this loop is probably unnecessary, but useful if you want to have some conditional on what trains are tracked
        #for example, only a particular route, or that will leave in more than x minutes, etc.

    trains = []
    for arr in trainsatstop:
        trains.append(arr)
    for train in trains:
        ETA = parseCTAtime(train["arrT"]) - parseCTAtime(train["prdt"])
        minutes_prelim = str(ETA)
        train["ETA"] = int(minutes_prelim[2:4])
        #calculates ETA by subtracting ETA from the time the estimate was generated
        #see appendix D of the CTA train API documentation

    output = [[],[]]
    for train in trains:
        if train["isSch"] == "1":
            trainstring = str(train["ETA"])+' minutes to ' + train["destNm"] +', scheduled'
        elif train["isDly"] == "1":
            trainstring = str(train["ETA"])+' minutes to ' + train["destNm"] +', delayed'
        else:
            trainstring = str(train["ETA"])+' minutes to ' + train["destNm"]
        #the above generates an output string for display.  It has the ETA in minutes, dest,...
        #...and whether the train is scheduled or delayed.
        #This is the string you'd normally want to display.
        #theoretically you could also spit out the raw data and have the HTML in the display fill it in.
        #
        #Now the output is split into two halves, for the two directions the CTA uses for rail services.
        #Each train gets the route, which can be used for showing the arrival in the correct color;
        #And the string above, followed by the run number, which is provision for use in transfer-calculation.
        if train["trDr"] == "1":
            output[0].append([train["rt"],train["rn"],trainstring])
        elif train["trDr"] == "5":
            output[1].append([train["rt"],train["rn"],trainstring])

    return output

In [27]:
teststop = "41320"
data = fetchCTAtrains(teststop)
CTAtraininfo(data)

http://lapi.transitchicago.com/api/1.0/ttarrivals.aspx?key=c103a1622f9b49e0b1eed52042bff7c2&mapid=41320&outputType=JSON


[[['Brn', '403', '2 minutes to Kimball'],
  ['Brn', '411', '6 minutes to Kimball'],
  ['Red', '801', '8 minutes to Howard'],
  ['Red', '825', '15 minutes to Howard'],
  ['Brn', '412', '17 minutes to Kimball']],
 [['Brn', '406', '2 minutes to Loop'],
  ['Red', '903', '2 minutes to 95th/Dan Ryan'],
  ['Red', '904', '7 minutes to 95th/Dan Ryan'],
  ['Brn', '417', '11 minutes to Loop']]]